In [137]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.2)

llm_smart = ChatOpenAI(model="gpt-4o", temperature=0.2)

In [18]:
import subprocess

subprocess.run(["ls"], cwd="./code") 

app.go
app_test.go
go.mod


CompletedProcess(args=['ls'], returncode=0)

In [139]:
from typing import Annotated
from langgraph.graph.message import AnyMessage, add_messages
from langchain_core.language_models import BaseChatModel
from langchain_core.prompts import ChatPromptTemplate
from typing_extensions import TypedDict

class State(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]
    
class CreateFunctionPlanner:
    def __init__(self, llm: BaseChatModel, prompt: ChatPromptTemplate):
        self.runnable = prompt | llm

    def __call__(self, state: State) -> dict:
        return {"messages": [self.runnable.invoke({"messages": state["messages"]})]}

In [151]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
create_function_plan_prompt = ChatPromptTemplate.from_messages([
      ("system",
      """
      You are professional golang developer that excel in TDD.
      You will be provided with high level requirements.
      Let's first understand provided requirements then create application blueprint.
      Please create plan for implementation of functions needed to complete the requirements.
      Each function should have a clear purpose and should be named descriptively.
      If any function is too complex, you can break it down into smaller functions.
      The output will be 2 parts, the first part will be the list of functions and
      the second part will be flow of how these functions will interact with each other.
      The first part output will start with "Functions:" and followed by this format,
      - Signature:
        Purpose:
      The second part output will start with "Flow:" followed by the flow of functions.
      After you get feedback on the plan, you will revise the plan based on the feedback.
      No preamble or explanation is needed, just the plan.
      """),
      ("user",
      """
      {messages}
      """),
  ])
def create_function_plan(requirement: str) -> str:
  chain = create_function_plan_prompt | llm | StrOutputParser()
  test_cases = chain.invoke({"input": requirement})
  return test_cases

In [ ]:
requirement = """
Given 2 list of objects, list A and B, that each object consist of key is_main, order, and image_id.
Return a list of change that need to be made to transform list A to list B.
"""

In [152]:

from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver

planner = CreateFunctionPlanner(llm, create_function_plan_prompt)
# Define a new graph
workflow = StateGraph(State)
workflow.add_node("planner", planner)
workflow.set_entry_point("planner")
workflow.add_edge("planner", END)

app = workflow.compile(checkpointer=MemorySaver())

In [153]:
from langchain_core.messages import HumanMessage
import uuid
thread_id = str(uuid.uuid4())
inputs = {"messages": [HumanMessage(content=requirement)]}
config = {"configurable": {"thread_id": thread_id}}
while True:
    for output in app.stream(inputs, config):
        # stream() yields dictionaries with output keyed by node name
        for key, value in output.items():
            print(f"Output from node '{key}':")
            print("---")
            print(value)
        print("\n---\n")
    snapshot = app.get_state(config)
    # If "next" is present, it means we've interrupted mid-execution
    if not snapshot.next:
        break
    inputs = None
    response = input(
        "Do you approve the next step? Type y if you do, anything else to stop: "
    )
    if response != "y":
        break

Output from node 'planner':
---
{'messages': [AIMessage(content='Functions:\n- Signature: findChanges(listA []Object, listB []Object) []Change\n  Purpose: Find and return a list of changes needed to transform list A to list B.\n\n- Signature: isMainChanged(objA Object, objB Object) bool\n  Purpose: Check if the is_main attribute of two objects is different.\n\n- Signature: isOrderChanged(objA Object, objB Object) bool\n  Purpose: Check if the order attribute of two objects is different.\n\n- Signature: isImageIDChanged(objA Object, objB Object) bool\n  Purpose: Check if the image_id attribute of two objects is different.\n\n- Signature: createChange(objA Object, objB Object) Change\n  Purpose: Create a Change object based on the differences between two objects.\n\n- Signature: applyChange(objA *Object, change Change)\n  Purpose: Apply the given change to the object A.\n\n- Signature: revertChange(objA *Object, change Change)\n  Purpose: Revert the given change from the object A.\n\nFlo

In [155]:
for output in app.stream({"messages": [HumanMessage(content="""Merge all is<something>Change into one function""")]}, config):
    # stream() yields dictionaries with output keyed by node name
    for key, value in output.items():
        print(f"Output from node '{key}':")
        print("---")
        print(value)
    print("\n---\n")

Output from node 'planner':
---
{'messages': [AIMessage(content='Functions:\n- Signature: findChanges(listA []map[string]interface{}, listB []map[string]interface{}) []map[string]interface{}\n  Purpose: Find and return a list of changes needed to transform list A to list B.\n\n- Signature: areAttributesChanged(objA map[string]interface{}, objB map[string]interface{}) map[string]interface{}\n  Purpose: Check if the attributes of two objects are different and return a map of changed attributes.\n\n- Signature: createChange(objA map[string]interface{}, objB map[string]interface{}) map[string]interface{}\n  Purpose: Create a Change object based on the differences between two objects.\n\n- Signature: applyChange(objA *map[string]interface{}, change map[string]interface{})\n  Purpose: Apply the given change to the object A.\n\n- Signature: revertChange(objA *map[string]interface{}, change map[string]interface{})\n  Purpose: Revert the given change from the object A.\n\nFlow:\n1. Iterate over

In [101]:
def read_file_to_string(filename):
  """Reads the entire content of a file into a single string.

  Args:
      filename: The path to the file to be read.

  Returns:
      A string containing the entire content of the file, or None if the file
      could not be opened.
  """
  try:
    with open(filename, "r") as f:
      return f.read()
  except FileNotFoundError:
    return None
  
# Example usage
data = read_file_to_string("./code/app.go")
if data:
  print(data)
else:
  print("File not found!")

package code

func GetChanges(listA, listB []map[string]interface{}) []map[string]interface{} {
	var changes []map[string]interface{}

	for _, itemA := range listA {
		found := false
		for _, itemB := range listB {
			if itemA["image_id"] == itemB["image_id"] {
				found = true
				if itemA["order"] != itemB["order"] || itemA["is_main"] != itemB["is_main"] {
					change := make(map[string]interface{})
					change["action"] = "update"
					change["id"] = itemB["image_id"]
					change["order"] = itemB["order"]
					changes = append(changes, change)
				}
				break
			}
		}
		if !found {
			change := make(map[string]interface{})
			change["action"] = "delete"
			change["id"] = itemA["image_id"]
			changes = append(changes, change)
		}
	}

	for _, itemB := range listB {
		found := false
		for _, itemA := range listA {
			if itemB["image_id"] == itemA["image_id"] {
				found = true
				break
			}
		}
		if !found {
			change := make(map[string]interface{})
			change["action"] = "add"
			chan

In [112]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate
def create_test_cases(requirement: str, test_cases: list[str]) -> str:
    create_test_cases_prompt = ChatPromptTemplate.from_messages([
        ("system",
        """
        You are professional golang developer that excel in TDD.
        You will be provided with requirements and existing test cases.
        Let's first understand provided requirements and create a new test case that not duplicate with existing test cases.
        Please output a test case in json format.
        The json will have following key: 'Name', 'Inputs' and 'Output'.
        The test case name should be in natural language and describe the test case.
        No preamble or explanation is needed, just the plan.
        """),
        ("user",
        """
        Requirement: {requirement}
        Existing test cases: {test_cases}
        """),
    ])
    chain = create_test_cases_prompt | llm | JsonOutputParser()
    test_case = chain.invoke({"requirement": requirement, "test_cases": test_cases})
    return test_case

In [126]:
test_cases = [
    """{'Name': 'A list with one object needs to be updated order to match B list', 'Inputs': {'ListA': [{'is_main': true, 'order': 1, 'image_id': 10}], 'ListB': [{'is_main': true, 'order': 2, 'image_id': 10}]}, 'Output': [{'action': 'update', 'image_id': 10, 'order': 2}]}""",
]

In [127]:
new_test_case = create_test_cases(requirement, test_cases)
test_cases.append(new_test_case)
for test_case in test_cases:
    print(test_case)

{'Name': 'A list with one object needs to be updated order to match B list', 'Inputs': {'ListA': [{'is_main': true, 'order': 1, 'image_id': 10}], 'ListB': [{'is_main': true, 'order': 2, 'image_id': 10}]}, 'Output': [{'action': 'update', 'image_id': 10, 'order': 2}]}
{'Name': 'A list with one object needs to be updated image_id to match B list', 'Inputs': {'ListA': [{'is_main': True, 'order': 1, 'image_id': 10}], 'ListB': [{'is_main': True, 'order': 1, 'image_id': 20}]}, 'Output': [{'action': 'update', 'image_id': 20, 'order': 1}]}


In [119]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
def create_unit_test_code(requirement: str, test_case: str, existing_code: str) -> str:
    create_unit_test_code_prompt = ChatPromptTemplate.from_messages([
        ("system",
        """
        You are professional golang developer that excel in TDD.
        Let's first understand provided requirements and a test case.
        You will be provided with a test case, expected result and existing test code that you need to modify to make it complete.
        Please output the unit test code from the given test case and existing test code, and name the test function as 'Test<test_case_name>'.
        No preamble or explanation is needed, just the code.
        """),
        ("user",
        """
        Requirement:
            {requirement}
        Test case:
            {test_case}
        Existing Test Code:
        <code>
            {existing_test_code}
        </code>
        """),
    ])
    chain = create_unit_test_code_prompt | llm | StrOutputParser()
    output = chain.invoke({"requirement": requirement,
                            "test_case": test_case,
                            "existing_test_code": existing_code})
    return output

In [128]:
test_case = test_cases[1]
print(create_unit_test_code(requirement, 
                            test_case, 
                            read_file_to_string("./code/app_test.go")))

```go
package code

import (
	"reflect"
	"testing"
)

func TestAListWithOneObjectNeedsToBeUpdatedImageIDToMatchBList(t *testing.T) {
	listA := []map[string]interface{}{{"is_main": true, "order": 1, "image_id": 10}}
	listB := []map[string]interface{}{{"is_main": true, "order": 1, "image_id": 20}}
	expectedOutput := []map[string]interface{}{{"action": "update", "image_id": 20, "order": 1}}

	result := GetChanges(listA, listB)

	if !reflect.DeepEqual(result, expectedOutput) {
		t.Errorf("Expected %v, but got %v", expectedOutput, result)
	}
}
```


In [129]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

def create_code(requirement: str, test_result: str, existing_code: str) -> str:
    create_code_prompt = ChatPromptTemplate.from_messages([
        ("system",
        """
        You are professional golang developer that excel in TDD.
        Let's first understand provided requirements.
        You will be provided with a test result, existing code that you need to modify to make it complete and pass all test cases.
        Please output the function code.
        No preamble or explanation is needed, just the plan.
        """),
        ("user",
        """
        Requirement:
            {requirement}
        Test result:
            {test_result}
        Code:
        <code>
            {code}
        </code>
        """),
    ])
    chain = create_code_prompt | llm | StrOutputParser()
    output = chain.invoke({"requirement": requirement,
                            "test_result": test_result,
                            "code": existing_code})
    return output

In [ ]:
test_result = """

"""
print(create_code(requirement, 
                  test_result, 
                  read_file_to_string("./code/app.go")))